In [31]:
import rasterio
import geopandas as gpd
import pyproj
import os
import matplotlib.pyplot as plt

In [32]:
def snow_depth(path, DEM, SLOPE, outdir):
    
    for file in os.listdir(path):
        if file.endswith('shp'):
            filename = file[0:5]
            points = os.path.join(path,file)

            pts = gpd.read_file(points)
            

            pts['lon'] = pts.geometry.x
            pts['lat'] = pts.geometry.y

            coords = [(x,y) for x, y in zip(pts.lon, pts.lat)]

            src = rasterio.open(DEM)
            slope = rasterio.open(SLOPE)
            water = rasterio.open(WATER)
            fc = rasterio.open(FC)
            wsc = rasterio.open(WSC)
            si = rasterio.open(SI)
            

            pts['DEM'] = [x[0] for x in src.sample(coords)]
            pts['SLOPE'] = [x[0] for x in slope.sample(coords)]
            pts['WATER'] = [x[0] for x in water.sample(coords)]
            pts['FC'] = [x[0] for x in fc.sample(coords)]
            pts['WSC'] = [x[0] for x in wsc.sample(coords)]
            pts['SI'] = [x[0] for x in si.sample(coords)]
            
            #TESTING ARCTIC DEM
            #pts['SD'] = round(pts.Est_Surf - pts.DEM,2)
            pts['SD'] = round(pts.Surf_mean - pts.DEM,2)
            
            print('Training Points: ',pts.shape[0])
            pts = pts[pts['SLOPE'] < 8]
            print('Slope: ',pts.shape[0])
            pts = pts[pts['WATER'] < 1]
            print('water: ',pts.shape[0])
            pts = pts[pts['FC'] > 0]
            print('fc: ',pts.shape[0])
            pts = pts[pts['WSC'] > 0]
            print('wsc: ',pts.shape[0])
            
            pts = pts[pts['SD'] < 8]
            pts = pts[pts['SD'] > 0]
            print('Outliers: ',pts.shape[0])

            pts.to_file(outdir+'SD_DL'+filename+'.shp')
            print('processing completed for beam: ', filename)
    return pts
        

In [33]:
DEM = 'C:/Users/Rasmu/Documents/Thesis/DEM/ArcticDEM_10m_DL.tif'
#DEM = 'C:/Users/Rasmu/Documents/Thesis/DEM/MergedDEM10m.tif'
SLOPE = 'C:/Users/Rasmu/Documents/Thesis/Slope_Mask/Slope.tif'
WATER = 'C:/Users/Rasmu/Documents/Thesis/Water_Mask/WATERMASK.tif'
FC = 'C:/Users/Rasmu/Documents/Thesis/S2_Masks/Forest_Cover_Mask.tif'
WSC = 'C:/Users/Rasmu/Documents/Thesis/Water_Mask/WSC_Mask.tif'
SI = 'C:/Users/Rasmu/Documents/Thesis/S2_Masks/SI_Geilo_100m.tif'
path = 'C:/Users/Rasmu/Documents/Thesis/Surface_Estimation/'
outdir = 'C:/Users/Rasmu/Documents/Thesis/Snow_Depth_2/'

In [34]:
df = snow_depth(path, DEM, SLOPE,  outdir)

Training Points:  12068
Slope:  3286
water:  2252
fc:  1714
wsc:  1397
Outliers:  687
processing completed for beam:  gt2l1


In [35]:
def Visual(df):

  plt.figure(figsize=(16, 8))
  y1 = 'Est_Surf'
  y2 = 'DEM'

  minVal1, maxVal1 = df[y1].min(skipna=True), df[y1].max()
  minVal2, maxVal2 = df[y2].min(skipna=True), df[y2].max()
  minVal3 = minVal2 - 10

  plt.scatter(df.AT_dist.values,df.Est_Surf.values,c='g',s=5)
  plt.scatter(df.AT_dist.values,df.DEM.values,c='r',s=5)

  #plt.ylim(minVal1, maxVal1);
  #plt.xlim(df.AT_dist.values[0],df.AT_dist.values[-1]);
  plt.ylim(1300, 1400);
  plt.xlim(67000,70500);

In [36]:
#Visual(df)

In [37]:
df.SD.std()

1.282904693572548

In [38]:
df.SD.mean()

2.5213245997088793